In [1]:
import pandas as pd
import os
import datetime
import shutil

In [3]:
file_list = os.listdir("..\\all\\")
file_list.sort()
dst_path = '..\\new_result\\'
src_path = '..\\all\\'
filename = ''

In [4]:
def read_file(system_id, ess_id):
    global filename
    filename = 'ess_' + system_id + '_' + ess_id + '.csv'
    df = pd.read_csv(src_path + filename, parse_dates=['C.target_date'], index_col = ['C.target_date'])
    return df

In [5]:
def accumulate_charge(df):
    df['accu_chg_e'] = df['M.batt1_chg'].cumsum()
    df['accu_dis_e'] = df['M.batt1_dis'].cumsum()
    df['chg_dis_ratio'] = df['accu_dis_e'].div(df['accu_chg_e']).mul(100)
    
    df['pv_e'] = df['M.string1_e'].add(df['M.string2_e'])
    df['accu_pv_e'] = df['pv_e'].cumsum()
    
    df['accu_sell_e'] = df['M.grid_e_sell'].cumsum()
    df['accu_buy_e'] = df['M.grid_e_buy '].cumsum()
    df['pv_sell_ratio'] = df['accu_sell_e'].div(df['accu_pv_e']).mul(100)

    df['accu_load_e'] = df['M.consumption_e'].cumsum()
    
    df.to_csv(dst_path + 'accumulate_' + filename)

In [6]:
for i in range(len(file_list)):
    system_id = file_list[i].split('_')[1]
    ess_id = file_list[i].split('_')[2].split('.')[0]
    df = read_file(system_id, ess_id)
    accumulate_charge(df)
    del df